## Files preparing

In [ ]:
import os
import shutil

def make_folders(test_size, path_with_classes):
    
    if os.path.exists("test"):
        shutil.rmtree("test")
        shutil.rmtree("train")
    
    if not os.path.exists("test"):
        os.makedirs("test")
        os.makedirs("train")
    
    classes = os.listdir(path_with_classes)
    for folder in classes: 
        test_name = "test/" + folder
        train_name = "train/" + folder 
        os.makedirs(test_name)
        os.makedirs(train_name)
        files = os.listdir(path_with_classes + "/" + folder)
        test_num = int(test_size * len(files))
        for ind in range(test_num):
            shutil.copy(path_with_classes+"/"+folder+"/"+files[ind], test_name+"/"+files[ind])
        for ind in range(test_num, len(files)):
            shutil.copy(path_with_classes+"/"+folder+"/"+files[ind], train_name+"/"+files[ind])

        
make_folders(0.3, "images")

## Necessary libraries

In [ ]:
from __future__ import print_function
from __future__ import division

from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Dense
from keras.layers import Input
import numpy as np

## Auxilary functions

In [21]:
def setup_generator(train_path, test_path, batch_size, dimentions):
    train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    validation_generator = test_datagen.flow_from_directory(
        test_path, # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    return train_generator, validation_generator


def load_image(img_path, dimentions, rescale=1. / 255):
    img = image.load_img(img_path, target_size=dimentions)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x *= rescale 

    return x


def get_classes(file_path):
    with open(file_path) as f:
        classes = f.read().splitlines()

    return classes


def create_model(num_classes, dropout, shape):
    base_model = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_tensor=Input(
            shape=shape))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model_final = Model(inputs=base_model.input, outputs=predictions)

    return model_final


def train_model(model_final, train_generator, validation_generator, callbacks, epochs_value, batch_size):
    model_final.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    model_final.fit_generator(train_generator, validation_data=validation_generator,
                              epochs=epochs_value, callbacks=callbacks,
                              steps_per_epoch=train_generator.samples//batch_size,
                              validation_steps=validation_generator.samples//batch_size)


def load_model(model_final, weights_path, shape):
    model_final = create_model(101, 0, shape)
    model_final.load_weights(weights_path)
    return model_final

## Constants

In [24]:
shape = (224, 224, 3)
batch_size = 32
dropout_value = 0.2
epochs_value = 3
image_path = ""
model_path = "saved_models/food-101-epoch-01.hdf5"


X_train, X_test = setup_generator('train', 'test', batch_size, shape[:2])

Found 2800 images belonging to 4 classes.
Found 600 images belonging to 2 classes.


## Building model

In [23]:
callbacks = []
callbacks.append(ModelCheckpoint(filepath='saved_models/weights.epoch-{epoch:02d}-val_loss-{val_loss:.2f}.hdf5',
                                       verbose=1, save_best_only=True))

model_final = create_model(X_train.num_class, dropout_value, shape)

In [5]:
train_model(model_final, X_train, X_test, callbacks, epochs_value, batch_size)

print("Training finished...")

Epoch 1/2
43/43 [==============================] - 3073s - loss: 1.0344 - acc: 0.6628 - val_loss: 8.0031 - val_acc: 0.5035
Epoch 2/2
43/43 [==============================] - 3174s - loss: 0.5376 - acc: 0.7975 - val_loss: 0.6937 - val_acc: 0.4930
Training finished...


OSError: Unable to open file (Unable to open file: name = 'saved_models/food-101-epoch-01.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

## Predictions

In [ ]:
trained_model = load_model(model_final, model_path, shape)
image = load_image(image_path, shape[:2])
preds = trained_model.predict(image)
classes = get_classes('meta/classes.txt')
print("the image is: ", classes([np.argmax(preds)]))